In [3]:
import pandas as pd
import json

In [4]:
df = pd.read_csv("/data/raw_stock_indicator.csv")

In [5]:
def json_parser(x):
    try:
        return json.loads(x)["symbol"]
    except Exception as e:
        print(e)
        return ""

In [6]:
df["company_symbol"] = df["info_json"].map(lambda x: json_parser(x))  

In [7]:
df["company_symbol"].unique()

array(['LTIM.NS', '20MICRONS.NS', '21STCENMGM.NS', '3MINDIA.NS',
       '3PLAND.NS', '5PAISA.NS', '63MOONS.NS', 'A2ZINFRA.NS', 'AAKASH.NS',
       'AARON.NS', 'AARTIDRUGS.NS', 'AARTIIND.NS', 'AARVEEDEN.NS',
       'AARVI.NS', 'AAVAS.NS', 'ABAN.NS', 'ABB.NS', 'POWERINDIA.NS',
       'ACC.NS', 'ACCELYA.NS', 'ACCURACY.NS', 'ACEINTEG.NS', 'ACE.NS',
       'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS',
       'ADANITRANS.NS', 'ADFFOODS.NS', 'ABCAPITAL.NS', 'ABFRL.NS',
       'BIRLAMONEY.NS', 'ADORWELD.NS', 'ADROITINFO.NS', 'ADVENZYMES.NS',
       'ADVANIHOTR.NS', 'AEGISCHEM.NS', 'AFFLE.NS', 'AGARIND.NS',
       'AGRITECH.NS', 'AGROPHOS.NS', 'ATFL.NS', 'AHLADA.NS',
       'AHLUCONT.NS'], dtype=object)

In [11]:
df.columns

Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA', 'Total Unusual Items',
       'Total Unusual Items Excluding Goodwill',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBIT',
       'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Otherunder Preferred Stock Dividend',
       'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Non Operating Income Expenses', 'Special Income Charges',
       'Restructuring And Mergern Acquisition',
       'Net Non Operating Interest Inco

In [15]:
cols = ["Total Revenue", "Operating Revenue", "Gross Profit", "Year", "Month", "company_symbol" ,"company_symbol"]

In [16]:
df1 = df[cols]

In [19]:
df1["time_lag"] = df1["Year"].str+df1["Month"].str

AttributeError: Can only use .str accessor with string values!